In [1]:
import pims
import matplotlib.pyplot as plt
import numpy as np
import trackpy as tp
import scipy.optimize as optimization
import pandas as pd
from scipy import asarray as ar,exp
import math
from mpl_toolkits.mplot3d import Axes3D

saveloc = "E:\\Stephan Bachelorproject\\20160627_long_3045.mdb\\features overnight\\"
location = "E:\\Stephan Bachelorproject\\20160627_long_3045.mdb\\images overnight\\"
True

True

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  4 10:32:39 2016

@author: Stephan
"""

def center_of_mass(particles):
    '''
    Calculate the center of mass
    All particles have same weigth
    '''
    total_x = 0
    total_y = 0
    total_z = 0
    count = 0
    for index, row in particles.iterrows():
        total_x += row['x']
        total_y += row['y']
        total_z += row['z']
        count += 1 
    cm_x = total_x / count
    cm_y = total_y / count
    cm_z = total_z / count
    return cm_x, cm_y, cm_z
    
def radius_of_gyration(particles, cm_x, cm_y, cm_z):   
    '''
    Calculate radius of gyration as
    Rg = sqrt(N / sum(r))
    with Rg: radius of gyration
    N: amount of particles
    r: distance to center of mass
    '''
    # calculate mean distance
    total_sum = 0
    count = 0
    for index, row in particles.iterrows():
        x = row['x'] 
        y = row['y']
        z = row['z']
        count += 1
        total_sum += (np.sqrt(x**2 + y**2 + z**2) - np.sqrt(cm_x**2 + cm_y**2 + cm_z**2))**2
    Rg = np.sqrt(total_sum / count)
    return Rg, count 
 
def particles_within(particles, radius, ofsetx, ofsety, ofsetz):
    '''
    Make a circle and find the particles that are inside this circle.
    '''
    return particles[( ((radius + ofsetx - particles['x'])**2 + (radius + ofsetz - particles['z'])**2 +
            (radius + ofsety - particles['y'])**2) < radius**2 )]
            
    
def fractal_fit(x, df, a):
    '''
    function used to find fractal dimension on a fit with
    N; partickles vs Radius of gyration
    '''
    return np.power(x / a ,df)        



In [37]:
class particle_class():
    def __init__(self, Id, xpos, ypos, zpos):
        self.identity = Id
        self.posx = xpos
        self.posy = ypos
        self.posz = zpos
        self.neigbour_list = []
    def get_x(self):
        return self.posx
    def get_y(self):
        return self.posy
    def get_z(self):
        return self.posz
    def add_neigbour(self, neigbour):
        self.neigbour_list.append(neigbour)
    def get_neigbour(self):
        return self.neigbour_list
    def get_id(self):
        return self.identity

def refine_features(features, item):
    global fBefore
    global fAfter
    global massCutoff
    global succes
    global failed
    
    fBefore.append(len(features))
    
    y2, x2 = np.histogram(features['mass'], bins = 1000)
    
    # delete all 0
    x2 = x2[1:]
    x=[]
    y=[]
    for i in range(len(x2)):
        if(y2[i] != 0):
            x.append(x2[i])
            y.append(y2[i])

    plt.scatter(x,y)
    plt.show()
    def gaus(x,a,x0,sigma, b, x1, sigma2):
        return a*exp(((-(x-x0)**2)/(2*sigma**2))) + b*exp(((-(x-x1)**2)/(2*sigma2**2)))


    try:
        popt, pcov = optimization.curve_fit(gaus,x,y, maxfev = 1000)


        x0 = popt[1]
        x1 = popt[4]
        if (x0 > x1):
            firstPeak = x1
            secondPeak = x0
        else:
            firstPeak = x0
            secondPeak = x1
            
        succes.append(item)

    except:
        print("failed to refine at ", item)
        failed.append(item)
        fAfter.append(len(features))
        return features.nlargest(1500,'mass')
        
    lowest = len(features)
    massMinimum = 0
    for i in range(0,len(y)):
        # check for upper lowest bound???
        if(y[i] < lowest and x[i] < secondPeak and  x[i] > firstPeak):
            lowest = y[i]
            massMinimum = x[i]

    massCutoff.append(massMinimum)
    features = features[features['mass'] < massMinimum*0.9]
    
    fAfter.append(len(features))
    return features

In [ ]:
'''
Find connectivity through time
'''
# init
particlesFoundList = []
meanNeigbourList = []
maxNeigbourList = []
minNeigbourList = []
histList = []
fBefore = []
fAfter = []
massCutoff = []
succes = []
failed =[]

# loop trough doList range
doList = [i for i in range(2,3)]
print(doList)

for item in doList:
    # Initialize
    histListPerTimeFrame = []
    
    # load located features
    features = pd.read_pickle(saveloc+'features_'+str(item)+'.pkl')
    print("loaded")
    
    # Refine features
    features = refine_features(features, item)
    print("refined")
    
    # add found to list
    particlesFound = len(features)
    particlesFoundList.append(particlesFound)

    # Rezise particles radius
    features['z'] = (features['z'])*0.6
    features['x'] = (features['x'] / 1024)*105.77
    features['y'] = (features['y'] / 1024)*105.77
    
    # define cutoff
    cutoff = 5.0 # micro meters

    # Create particles class
    particles = []
    for index, row in features.iterrows():
        newparticle = particle_class(index, row['x'], row['y'], row['z'])
        particles.append(newparticle)

    # calculate connectivity 
    total = 0
    Max = 0
    Min = 100
    total_dis = []
    
    print("Particles created, start loop")
    # loop over all particles
    for particle in particles:      
        # first find all neigbours
        for neigbour in particles:  
            # dont add yourself as neibour
            if (neigbour.get_id() != particle.get_id()):            
                distance = (math.pow(particle.get_x() - neigbour.get_x(),2) + 
                            math.pow(particle.get_y() - neigbour.get_y(),2) + 
                            math.pow(particle.get_z() - neigbour.get_z(),2))
                
                # check if between cutoff
                if(math.sqrt(distance) < cutoff):
                    total_dis.append(math.sqrt(distance))
                    particle.add_neigbour(neigbour)
                    total+=1
        if (len(particle.get_neigbour()) > Max):
            Max = len(particle.get_neigbour())
        if (len(particle.get_neigbour()) < Min):
            Min = len(particle.get_neigbour())
        
                
    meanTotal = 0
    for particle in particles:
        amountNeigbours = len(particle.get_neigbour())
        meanTotal += amountNeigbours
        histListPerTimeFrame.append(amountNeigbours)
    histList.append(histListPerTimeFrame)
    meanNeigbourList.append(meanTotal / particlesFound)
    maxNeigbourList.append(Max)
    minNeigbourList.append(Min)


True

[2]
loaded
refined
Particles created, start loop


In [ ]:
"""
3D plot of all features.
"""

plt.clf()
plt.close()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
xs = features['x']
ys = features['y']
zs = features['z']
ax.scatter(xs, ys, zs, c='b', marker='.')


ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_zlim(-52,53)
plt.show()

In [ ]:
"""
plot clusters
"""

import sys
sys.setrecursionlimit(10**6)

def search(p, neigbours, xs, ys, zs):
    global prev
    global xs2
    global ys2
    global zs2 
    
    if(len(neigbours) > 0):
        for n in neigbours:
            if n.get_id() not in prev:
                xs.append(n.get_x())
                ys.append(n.get_y())
                zs.append(n.get_z())
                prev.append(n.get_id())
                
                xs2.append( [n.get_x(),p.get_x()] )
                ys2.append( [n.get_y(),p.get_y()] )
                zs2.append( [n.get_z(),p.get_z()] )
                
                search(n, n.get_neigbour(), xs, ys, zs)
            else:
                xs2.append( [n.get_x(),p.get_x()] )
                ys2.append( [n.get_y(),p.get_y()] )
                zs2.append( [n.get_z(),p.get_z()] )
            

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import random

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

prev = []
for p in particles:
    if p.get_id() not in prev:
        xs = []
        ys = []
        zs = []
        prev.append(p.get_id())

        xs2 = []
        ys2 = []
        zs2 = []
        
        xs.append(p.get_x())
        ys.append(p.get_y())
        zs.append(p.get_z())
        search(p, p.get_neigbour(), xs, ys, zs)
        
        
        randomColor = random.choice(list(colors.cnames.items()))[1]
        for i in range(0, len(xs2)):
            ax.plot(xs2[i], ys2[i], zs2[i], c=randomColor)
        ax.scatter(xs, ys, zs, c=randomColor)
        
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
# ax.set_xlim(0,20)
# ax.set_ylim(0,20)
# ax.set_zlim(0,20)
print("done")
plt.show()

In [ ]:
"""
fast 3d plot
"""

import sys
sys.setrecursionlimit(10**6)

def search(p, neigbours, xs, ys, zs):
    global prev
    global xs2
    global ys2
    global zs2 
    
    if(len(neigbours) > 0):
        for n in neigbours:
            if n.get_id() not in prev:
                xs.append(n.get_x())
                ys.append(n.get_y())
                zs.append(n.get_z())
                ax.text(n.get_x(),n.get_y(),n.get_z(),  '%s' % str(len(n.get_neigbour())), size=20, zorder=1,  
                        color='k') 
                xs2.append( [n.get_x(),p.get_x()] )
                ys2.append( [n.get_y(),p.get_y()] )
                zs2.append( [n.get_z(),p.get_z()] )
                    
                prev.append(n.get_id())
                
                
                
                search(n, n.get_neigbour(), xs, ys, zs)
            else:
                xs2.append( [n.get_x(),p.get_x()] )
                ys2.append( [n.get_y(),p.get_y()] )
                zs2.append( [n.get_z(),p.get_z()] )

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as colors
import random

plt.clf()
plt.close()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100

prev = []
for p in particles:
    if p.get_id() not in prev:
        xs = []
        ys = []
        zs = []
        prev.append(p.get_id())

        xs2 = []
        ys2 = []
        zs2 = []
        
        xs.append(p.get_x())
        ys.append(p.get_y())
        zs.append(p.get_z())
        ax.text(p.get_x(),p.get_y(),p.get_z(),  '%s' % str(len(p.get_neigbour())), size=20, zorder=1,  
 color='k') 
        search(p, p.get_neigbour(), xs, ys, zs)
        
        
        randomColor = random.choice(list(colors.cnames.items()))[1]
        for i in range(0, len(xs2)):
            ax.plot(xs2[i], ys2[i], zs2[i], c=randomColor)
        ax.scatter(xs, ys, zs, c=randomColor)

        
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
# ax.set_xlim(0,20)
# ax.set_ylim(0,20)
# ax.set_zlim(0,20)
print("done")
plt.show()

In [ ]:
i = 1
features = pd.read_pickle(saveloc+ "features_" + str(i) + ".pkl")
image = np.load(location + 'image_' + str(i) + '.npy')
True


In [ ]:
print(len(features))
plt.hist(features['mass'], bins = 1000)
plt.show()

In [ ]:
image = np.load(location + 'image_' + str(70) + '.npy')
features = features[features['mass'] < 0.0]

In [ ]:
tp.annotate3d(features, image)

In [ ]:
tp.annotate3d(features, image)

In [ ]:
'''
Delete noise from features
use 2 gausians plots to find the two peaks, 
find minimum between these peaks, 
delete all features before with an error of 10 %
'''

import math 
from scipy import asarray as ar,exp

plt.figure()
histList = plt.hist(features['mass'], bins = 1000)
y = histList[0]
x = histList[1][:len(y)]
plt.clf()

def gaus(x,a,x0,sigma, b, x1, sigma2):
    return a*exp(((-(x-x0)**2)/(2*sigma**2))) + b*exp(((-(x-x1)**2)/(2*sigma2**2)))


popt, pcov = optimization.curve_fit(gaus,x,y, maxfev = 10000)


x0 = popt[1]
x1 = popt[4]
if (x0 > x1):
    firstPeak = x1
    secondPeak = x0
else:
    firstPeak = x0
    secondPeak = x1
    
lowest = len(features)
massMinimum = 0
for i in range(0,len(y)):
    # check for upper lowest bound???
    if(y[i] < lowest and x[i] < secondPeak and  x[i] > firstPeak):
        lowest = y[i]
        massMinimum = x[i]
        
        
plt.plot(x,y,'b+:',label='data')
plt.plot(x,gaus(x,*popt),'ro:',label='fit')
plt.errorbar(massMinimum, lowest, xerr=massMinimum*0.4,
             color='black', fmt='.', markersize='20', ecolor='black',capsize=4, label="Noise cutoff")
plt.legend()
plt.title('2 Gausian plots to find noise cutoff')
plt.xlabel('Mass')
plt.ylabel('Count')
plt.xlim(0,5)
plt.show()

In [ ]:
popt



In [ ]:

histList = plt.hist(features['mass'], bins = 1000)
countList = histList[0]
massList = histList[1]

lowest = len(features)
massMinimum = 0
for i in range(0,len(countList)):
    # check for upper lowest bound???
    if(countList[i] < lowest and massList[i] < 2.5):
        lowest = countList[i]
        massMinimum = massList[i]

print(massMinimum)
features = features[features['mass'] > (massMinimum*0.7)]
plt.clf()

In [ ]:

#print("Rezise particles radius")
features['z'] = (features['z'])*0.6
features['x'] = (features['x'] / 1024)*105.77
features['y'] = (features['y'] / 1024)*105.77        

#update
#print("found " + str(len(features)) + " paricles!")
pfound.append(len(features))

max_radius = int(40*0.6) # Edit this to max z heigth of measurement

for rad in range(max_radius, 3,-1):
    for width in range(-105,105,20):     
        ofsetx = max_radius - rad - width#+ ofset_x_start
        ofsety = max_radius - rad - width#+ ofset_y_start
        ofsetz = max_radius - rad
        # in 3d
        particles_in_rad = particles_within(features, rad, ofsetx, ofsety, ofsetz)

        if(particles_in_rad.empty == False):

            # calculate center of mass
            cm_x, cm_y, cm_z = center_of_mass(particles_in_rad)

            # calculate mean distance
            Rg, N = radius_of_gyration(particles_in_rad, cm_x, cm_y, cm_z)  

            if(N > 2):
                # only when more then 10 particles
                Rgs.append(Rg)
                Ns.append(N) 
                    



In [9]:
features = pd.read_pickle(saveloc+'features_'+str(8)+'.pkl')

In [30]:
plt.hist(features['mass'], bins = 1000)
plt.show()